In [29]:
from prefect.utilities.logging import get_logger
from prefect import task, Flow
import requests
import logging
import prefect
import dask
from dask import distributed
from time import time
from pprint import pprint
print(f"dask=={dask.__version__} prefect=={prefect.__version__}")   


dask==2022.05.0 prefect==1.2.1


In [30]:
class DaskHandler(prefect.utilities.logging.StreamHandler):
    def __init__(self, prefix):
        self._worker = None
        self._prefix = prefix

    def emit(self, record):
        if self._worker is None:
            self._worker = distributed.get_worker()
        self._worker.log_event(self._prefix, {"msg": record.msg})
        # requests.post("http://172.30.46.210:8000/", params=dict(msg=record.msg))


In [31]:
cluster=distributed.SSHCluster(["localhost","localhost"], connect_options={"known_hosts":None},silence_logs=False,worker_options={"data":{"nodelist":["a","b"]}})
client=distributed.Client(cluster)

2022-05-24 17:20:32,666 - distributed.deploy.ssh - DEBUG - Created Scheduler Connection
2022-05-24 17:20:34,025 - distributed.deploy.ssh - INFO - 2022-05-24 17:20:34,023 - distributed.scheduler - INFO - State start
2022-05-24 17:20:34,031 - distributed.deploy.ssh - INFO - 2022-05-24 17:20:34,029 - distributed.scheduler - INFO - Clear task state
2022-05-24 17:20:34,033 - distributed.deploy.ssh - INFO - 2022-05-24 17:20:34,031 - distributed.scheduler - INFO -   Scheduler at: tcp://192.168.191.100:37149
2022-05-24 17:20:34,035 - distributed.deploy.ssh - DEBUG - 2022-05-24 17:20:34,031 - distributed.scheduler - INFO -   Scheduler at: tcp://192.168.191.100:37149

2022-05-24 17:20:34,038 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:20:34,039 - distributed.comm.tcp - DEBUG - Setting TCP user timeout: 30000 ms
2022-05-24 17:20:34,048 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:20:

In [32]:
executor=prefect.executors.DaskExecutor(address=cluster.scheduler.address,debug=True)


In [33]:
@prefect.task(name="hello_world", log_stdout=True)
def hello_world():
    print("Hello world!")
    # distributed.get_worker().log_event("runtime", { "msg":"I'm hello world!"})
    logger = prefect.context.get("logger")
    logger.info("I'm hello world!")

    return "balala"


@prefect.task(name="foo", log_stdout=True)
def foo(msg):
    # worker = distributed.get_worker()
    # data = worker.data["nodelist"]
    # worker.log_event("runtime", {"time": time(), "nodelist": data})
    logger=distributed.get_worker().data.get("logger", None)
    if logger is None:
        logger = prefect.context.get("logger")
        logger.addHandler(DaskHandler("Foo"))
        distributed.get_worker().data["logger"] = logger

    logger.info("Just a test  {}".format(msg))
    # logger.removeHandler()
    pprint(logging.Logger.manager.loggerDict.keys())
    print(f"Foo! {msg}")


with prefect.Flow(name="flow_test") as flow:
    foo(hello_world())

# now attach our custom handler to Task B's logger


[2022-05-24 17:20:46+0800] INFO - prefect.FlowRunner | Beginning Flow run for 'flow_test'
[2022-05-24 17:20:46+0800] INFO - prefect.DaskExecutor | Connecting to an existing Dask cluster at tcp://192.168.191.100:37149


2022-05-24 17:20:46,799 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:20:46,803 - distributed.comm.tcp - DEBUG - Setting TCP user timeout: 30000 ms
2022-05-24 17:20:46,813 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:20:46,815 - distributed.comm.tcp - DEBUG - Setting TCP user timeout: 30000 ms
2022-05-24 17:20:46,825 - distributed.client - DEBUG - Started scheduling coroutines. Synchronized
2022-05-24 17:20:46,840 - distributed.client - DEBUG - Submit _maybe_run(...), hello_world-b1061b78e2e545afbd8b392bd5aab51c
2022-05-24 17:20:46,856 - distributed.client - DEBUG - Submit _maybe_run(...), foo-f22382b7f17349a7ba0798895a1458df
2022-05-24 17:20:46,863 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:20:46,864 - distributed.comm.tcp - DEBUG - Setting TCP user timeout: 30000 ms
2022-05-24 17:20:46,866 - distributed.client - DE

[2022-05-24 17:20:47+0800] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">

In [35]:
pprint(logging.Logger.manager.loggerDict.keys())


dict_keys(['concurrent.futures', 'concurrent', 'asyncio', 'tornado.access', 'tornado', 'tornado.application', 'tornado.general', 'prompt_toolkit.buffer', 'prompt_toolkit', 'parso.python.diff', 'parso.python', 'parso', 'parso.cache', 'IPKernelApp', 'prefect', 'prefect.NoResultType', 'urllib3.util.retry', 'urllib3.util', 'urllib3', 'urllib3.connection', 'urllib3.response', 'urllib3.connectionpool', 'urllib3.poolmanager', 'charset_normalizer', 'requests', 'prefect.FlowRunner', 'prefect.backend.flow_run', 'prefect.backend', 'prefect.backend.flow', 'distributed.config', 'distributed', 'distributed.client', 'bokeh', 'distributed.worker', 'distributed.utils', 'fsspec', 'distributed.protocol.compression', 'distributed.protocol', 'distributed.protocol.pickle', 'distributed.protocol.core', 'distributed.comm.core', 'distributed.comm', 'distributed.comm.utils', 'distributed.comm.inproc', 'distributed.comm.tcp', 'distributed.comm.ws', 'distributed.diagnostics.plugin', 'distributed.diagnostics', 'di

In [36]:
client=distributed.Client(cluster)

2022-05-24 17:20:58,336 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:20:58,337 - distributed.comm.tcp - DEBUG - Setting TCP user timeout: 30000 ms
2022-05-24 17:20:58,342 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:20:58,343 - distributed.comm.tcp - DEBUG - Setting TCP user timeout: 30000 ms
2022-05-24 17:20:58,356 - distributed.client - DEBUG - Started scheduling coroutines. Synchronized


In [43]:
flow.run(executor= executor)


[2022-05-24 17:21:38+0800] INFO - prefect.FlowRunner | Beginning Flow run for 'flow_test'
[2022-05-24 17:21:38+0800] INFO - prefect.DaskExecutor | Connecting to an existing Dask cluster at tcp://192.168.191.100:37149


2022-05-24 17:21:38,866 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:21:38,869 - distributed.comm.tcp - DEBUG - Setting TCP user timeout: 30000 ms
2022-05-24 17:21:38,878 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:21:38,882 - distributed.comm.tcp - DEBUG - Setting TCP user timeout: 30000 ms
2022-05-24 17:21:38,893 - distributed.client - DEBUG - Started scheduling coroutines. Synchronized
2022-05-24 17:21:38,907 - distributed.client - DEBUG - Submit _maybe_run(...), hello_world-a7a47eeb67094ca4bb024959cddcbfac
2022-05-24 17:21:38,922 - distributed.client - DEBUG - Submit _maybe_run(...), foo-491e044e837340fdad810a04f87ec1ca
2022-05-24 17:21:38,928 - distributed.client - DEBUG - Waiting on futures to clear before gather
2022-05-24 17:21:38,934 - distributed.comm.tcp - DEBUG - Setting TCP keepalive: nprobes=10, idle=10, interval=2
2022-05-24 17:21:38,936 - distributed.comm.t

[2022-05-24 17:21:39+0800] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">

In [44]:
pprint(client.get_events("prefect"))

((1653384047.1904552, {'msg': 'Just a test  balala'}),
 (1653384073.0968456, {'msg': 'Just a test  balala'}),
 (1653384085.6051018, {'msg': 'Just a test  balala'}),
 (1653384092.1818671, {'msg': 'Just a test  balala'}),
 (1653384098.9724736, {'msg': 'Just a test  balala'}))


Future exception was never retrieved
future: <Future finished exception=TimeoutError('Timeout')>
tornado.util.TimeoutError: Timeout
Future exception was never retrieved
future: <Future finished exception=TimeoutError('Timeout')>
tornado.util.TimeoutError: Timeout


In [ ]:
with executor.start():
    future=executor.submit(boo,"what's up")
    res=executor.wait([future])
    pprint(f"result = {res}")
    pprint(executor.client.get_events("runtime"))

In [28]:
cluster.close()


In [ ]:
executor.client.get_events("runtime")

In [ ]:
prefect.engine.get_default_flow_runner_class()

In [ ]:
cluster.workers[0].status

In [ ]:
prefect.engine.get_default_task_runner_class()

In [ ]:
@task(name="Task A")
def task_a():
    return 3


@task(name="Task B")
def task_b(x):
    logger = prefect.context.get("logger")
    logger.info("Beginning to run Task B with input {}".format(x))
    y = 3 * x + 1
    logger.info("Returning the value {}".format(y))
    return y


with Flow("logging-example") as flow:
    result = task_b(task_a)


# now attach our custom handler to Task B's logger
task_logger = get_logger("Task B")
task_logger.addHandler(MyHandler())


In [ ]:
flow.run()

In [ ]:
requests.post("http://172.30.46.210:8000/", params=dict(msg="Hello world!"))

In [ ]:
class FooTask(Task):
    def __init__(self, *args, **kwargs):
        self._cmd = ....
        pass

    def get_config(self):
        pass

    def run_raw(self):
        self.get_config()
        subprocess.exec(self._cmd)

    def run(self):
        try:
            self.run_raw()
        except RuntimeWarning as run_warning:
            logger.warning(run_error)
        except RuntimeError as run_error:
            logger.error(run_error)
        except Exception as error:
            logger.err(error)
        else:
            pass
